In [2]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, load_in_8bit=True, device_map='auto')

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/mymusise


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
input_text = "假设你现在是一个游戏策划，现在你要设计一款生存类的RPG游戏，请帮忙设计这款游戏的世界机制："

ids = tokenizer.encode(input_text, return_tensors='pt')

outs = model.generate(ids, max_length=128)

response = tokenizer.batch_decode(outs)
print(response[0])

/home/mymusise/pro/ChatGLM-Tuning/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The dtype of attention mask (torch.int64) is not bool


假设你现在是一个游戏策划,现在你要设计一款生存类的RPG游戏,请帮忙设计这款游戏的世界机制: 设计一款生存类的RPG游戏,可以参考下述世界机制:
1. 世界背景:游戏的故事背景可以设置在一个虚构的世界中,比如现代都市、未来世界、古代文明等等。世界的背景可以包括各种元素,比如科技、环境、政治、文化等等。
2. 地图:游戏地图可以是一个大型的虚拟世界,包含各种地形、地貌、城市、森林、沙漠等等。地图可以包含各种资源点、任务点、战斗点等等。
3.


In [3]:
from peft import LoraConfig, TaskType, prepare_model_for_int8_training, get_peft_model

model.gradient_checkpointing_enable()
model.is_parallelizable = True
model.model_parallel = True
model.config.use_cache = (
    False  # silence the warnings. Please re-enable for inference!
)

prepare_model_for_int8_training(model, output_embedding_layer_name='score')
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)

In [4]:
datas = [
{
    "context": "大师，我最近事业不顺利，感觉很迷茫，该怎么办？",
    "target": "孩子，生活中的起伏和挫折是常态。放下烦恼，用平静的心态面对困境，努力奋斗，同时保持内心的善良与慈悲，你将找到属于你的光明之路。",
},
{
    "context": "师父，我与家人之间常常发生争吵，如何改善家庭关系？",
    "target": "阿弥陀佛，亲情是一份宝贵的财富，要从宽容和理解开始。培养平和的心态，倾听彼此的意见，以慈悲与智慧去化解冲突，共同创造和谐温暖的家庭氛围。",
},
{
    "context": "师傅，我经常感到焦虑和压力，怎样才能获得内心的宁静？",
    "target": "孩子，内心的宁静源于觉察和自我调适。修行冥想，深呼吸，寻找内心的栖息地。明白外在的变幻无常，放下过多的执着与欲望，你将发现内心的平静与安宁。",
},
{
    "context": "大师，我常常陷入烦恼和抱怨，怎样才能培养感恩之心？",
    "target": "施主，感恩是一种修养，需要从日常小事开始。每天意识到自己的幸运和获得，体察他人的善意和关爱。珍惜当下，感恩生活中的点滴，你将发现更多的快乐与满足。",
},
{
    "context": "师父，我对未来感到迷茫和恐惧，不知道自己的人生目标是什么，有何建议？",
    "target": "阿弥陀佛，人生的方向不一定要一目了然，它是通过不断尝试和探索而逐渐清晰的。保持对未知的勇敢与开放，寻找自己内心的声音。用心去体验，用智慧去选择，你将找到属于自己的道路。",
}]

In [5]:
for item in datas:
    text = f"问:{item['context']}\n答:"
    ids = tokenizer.encode(text, return_tensors='pt')
    outs = model.generate(input_ids=ids, max_length=128)
    print(tokenizer.batch_decode(outs)[0])

/home/mymusise/pro/ChatGLM-Tuning/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


问:大师,我最近事业不顺利,感觉很迷茫,该怎么办?
答: 

首先,我很抱歉听到您的事业不顺利和迷茫的情况。如果您愿意,我可以提供一些建议,以帮助您重新振作起来并找到方向。

1. 接受现实并寻求支持:如果您已经尝试了各种方法来改善您的事业,但情况没有得到改善,那么接受现实可能是必要的。不要让自己陷入沮丧和绝望的情绪中,而是寻求支持,例如与家人、朋友或专业人士交谈。

2. 探索新的机会:如果您感到迷茫,那么探索新的机会可能是一个不错的选择。您可以考虑
问:师父,我与家人之间常常发生争吵,如何改善家庭关系?
答: 家庭关系是人生中非常重要的一部分,而争吵是家庭中常见的问题。以下是一些可能有助于改善家庭关系的建议:

1. 沟通:沟通是解决冲突的关键。尝试与家人坦诚地交流,听取他们的观点,并尝试理解对方的立场。在交流过程中,尽量避免指责和攻击,而是尝试寻找共同点和解决问题的方法。

2. 尊重:尊重是建立和谐关系的基础。尊重家人的感受和观点,并尝试接受他们的想法和决定。即使不同意他们的观点,也要尊重他们的权利和
问:师傅,我经常感到焦虑和压力,怎样才能获得内心的宁静?
答: 焦虑和压力是常见的情绪体验,但可以通过一些方法来缓解它们。以下是一些建议:

1. 深呼吸:深呼吸可以帮助你放松身体和心灵。尝试缓慢地吸气,然后慢慢地呼气,重复几次。

2. 冥想:冥想是一种放松和集中注意力的方法。找一个安静的地方,坐下来,闭上眼睛,专注于呼吸或一个特定的冥想练习。

3. 运动:运动可以帮助你释放紧张情绪和压力,并提高身体的代谢率。尝试进行一些轻度的运动,如散步
问:大师,我常常陷入烦恼和抱怨,怎样才能培养感恩之心?
答: 感恩之心是一种积极的情感,可以帮助我们更好地珍惜身边的人和事,并在生活中获得更多的快乐和满足感。以下是一些培养感恩之心的建议:

1. 意识到自己的感恩之心:要意识到感恩是一种积极的情感,而不是一种消极的情绪。当意识到自己正在感恩时,就可以开始思考身边的美好事物,并将它们纳入自己的行动计划中。

2. 感恩身边的人:感恩不仅仅是针对家人和亲人,还可以包括朋友、同事、邻居、社区和其他人。要时刻注意身边的美好事物,并表达感激之情。

3
问:师父,我对未来感到迷茫和恐惧,不知道自己的人生目标是什么,有何建议?
答: 对未来感到迷茫和恐惧是非常常见的感受,尤其是当

In [6]:
from tokenize_dataset_rows import preprocess
from datasets import Dataset

dataset = [preprocess(tokenizer, model.config, item, max_seq_length=256) for item in datas]

dataset = Dataset.from_list(dataset)

In [7]:
from finetune import ModifiedTrainer, data_collator
from transformers import TrainingArguments

training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 5,
    learning_rate = 1e-4,
    num_train_epochs=80,
    logging_steps=10,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/mymusise/pro/ChatGLM-Tuning/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mymusise. Use `wandb login --relogin` to force relogin


/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,4.195800
20,3.042800
30,1.922600
40,0.996700
50,0.398000
60,0.157500
70,0.089900
80,0.069700


TrainOutput(global_step=80, training_loss=1.3591203719377518, metrics={'train_runtime': 59.4406, 'train_samples_per_second': 6.729, 'train_steps_per_second': 1.346, 'total_flos': 853117835673600.0, 'train_loss': 1.3591203719377518, 'epoch': 80.0})

In [12]:
import torch

model.config.use_cache = (
    True
)

for item in datas:
    text = f"问:{item['context']}\n答:"
    ids = tokenizer.encode(text, return_tensors='pt')
    outs = model.generate(input_ids=ids, max_length=128)
    print(tokenizer.batch_decode(outs)[0])

/home/mymusise/pro/ChatGLM-Tuning/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


问:大师,我最近事业不顺利,感觉很迷茫,该怎么办?
答: 孩子,生活中的起伏和挫折是常态。放下烦恼,用平静的心态面对困境,努力奋斗,同时保持内心的善良与慈悲,你将找到属于你的光明之路。
问:师父,我与家人之间常常发生争吵,如何改善家庭关系?
答: 阿弥陀佛,亲情是一份宝贵的财富,要从宽容和理解开始。培养平和的心态,倾听彼此的意见,以慈悲与智慧去化解冲突,共同创造和谐温暖的家庭氛围。
问:师傅,我经常感到焦虑和压力,怎样才能获得内心的宁静?
答: 孩子,内心的宁静源于觉察和自我调适。修行冥想,深呼吸,寻找内心的栖息地。明白外在的变幻无常,放下过多的执着与欲望,你将发现内心的平静与安宁。
问:大师,我常常陷入烦恼和抱怨,怎样才能培养感恩之心?
答: 施主,感恩是一种修养,需要从日常小事开始。每天意识到自己的幸运和获得,体察他人的善意和关爱。珍惜当下,感恩生活中的点滴,你将发现更多的快乐与满足。
问:师父,我对未来感到迷茫和恐惧,不知道自己的人生目标是什么,有何建议?
答: 阿弥陀佛,人生的方向不一定要一目了然,它是通过不断尝试和探索而逐渐清晰的。保持对未知的勇敢与开放,寻找自己内心的声音。用心去体验,用智慧去选择,你将找到属于自己的道路。


In [ ]:
from peft.peft_model import PeftModelForCausalLM
type(model)

peft.peft_model.PeftModelForCausalLM

In [ ]:
examples = [
    """原句子：一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色，而茎上浓绿的叶子形成了生机盎然的对比。
人：帮我将百合花改成玫瑰花
修改后：一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色，而茎上浓绿的叶子形成了生机盎然的对比。""",

"""原句子：一朵充满生机的仙人掌花盛开着，它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。
人：请把花瓣涂成紫色
修改后：一朵充满生机的仙人掌花盛开着，它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。""",

"""原句子：一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔，站在一个充满多彩花朵的花园中。背景是蓝天和蓬松的云朵。
人：请将多彩花朵替换为充满活力的花朵。
修改后：一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔，站在一个充满充满活力的花朵的花园中。背景是蓝天和蓬松的云朵。""",

"""原句子：两个演员在一场京剧表演中进行激动人心的剑术对决。他们的动作流畅而精确，以他们对这门艺术的掌握能够吸引观众。
人：删除剑术对决的场景。
修改后：两个演员在一场京剧表演中展现他们流畅而精确的动作，以他们对这门艺术的掌握能够吸引观众。"""
]